In [1]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 3.4MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 3.5MB/s 


In [2]:
import numpy as np
import pandas as pd

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score


import eli5

from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd /content/drive/My Drive/Colab Notebooks/dataworkshop_matrix_two/dataworkshop_matrix_cat

/content/drive/My Drive/Colab Notebooks/dataworkshop_matrix_two/dataworkshop_matrix_cat


##Wczytywanie danych

In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [5]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [6]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [7]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [0]:
df = df[ df['price_currency'] != 'EUR' ]

##Przygotowanie cech

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:    
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [12]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [13]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [15]:
model2 = DecisionTreeRegressor(max_depth=5)
model2.fit(X, y)

imp = PermutationImportance(model2).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2562 ± 0.0018,param_napęd__cat
0.2036 ± 0.0059,param_faktura-vat__cat
0.1944 ± 0.0068,param_stan__cat
0.1445 ± 0.0092,param_rok-produkcji__cat
0.0646 ± 0.0035,param_moc__cat
0.0416 ± 0.0015,feature_kamera-cofania__cat
0.0413 ± 0.0007,param_skrzynia-biegów__cat
0.0276 ± 0.0046,param_marka-pojazdu__cat
0.0188 ± 0.0025,param_pojemność-skokowa__cat
0.0163 ± 0.0002,feature_bluetooth__cat
